In [18]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from binance import Client
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import sys
import ta
sys.path.insert(1, '/Users/arsenchik/Desktop/dipploma/machine_learning_in_hft/algotrading/backtesting')
from backtest import * 
from strategies import *
from config import *
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')


In [11]:
url = config.DATABASE_URL_1MINUTE
time_frame = config.INTERVAL_1MINUTE
td_days = 0

In [12]:
client = Client()
engine = create_engine(url=url)

In [37]:
# with engine.connect() as connection:
#     query = "SELECT name FROM sqlite_schema WHERE type='table'"
#     symbols = [item[0] for item in connection.execute(text(query)).fetchall()]
#     # for symbol in symbols:
#     #     qry = f"SELECT * FROM '{symbol}' WHERE Open_Time < '{pd.to_datetime('today') - timedelta(days = td_days)}'"
#     #     data = pd.DataFrame(connection.execute(text(qry))).set_index("Open_Time")
#     #     data.index = pd.to_datetime(data.index)
#     symbol = symbols[0]
#     qry = f"SELECT * FROM '{symbol}' WHERE Open_Time < '{pd.to_datetime('today') - timedelta(days = td_days)}'"
#     data = pd.DataFrame(connection.execute(text(qry))).set_index("Open_Time")
#     data.index = pd.to_datetime(data.index)

# connection.close()

# print(symbol)
# data

In [4]:
# data.plot(style='.',
#         figsize=(15, 5),
#         color=color_pal[0],
#         title='SOLUSDT')
# plt.show()

In [42]:
from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits=5)
data = data.sort_index()

In [5]:
# fig, axs = plt.subplots(5, 1, figsize=(15, 15), sharex=True)

# fold = 0
# for train_idx, val_idx in tss.split(df):
#     train = data.iloc[train_idx]
#     test = data.iloc[val_idx]
#     train['Close'].plot(ax=axs[fold],
#                           label='Training Set',
#                           title=f'Data Train/Test Split Fold {fold}')
#     test['Close'].plot(ax=axs[fold],
#                          label='Test Set')
#     axs[fold].axvline(test.index.min(), color='black', ls='--')
#     fold += 1
# plt.show()

In [15]:
def get_data(symbol):
    df = pd.DataFrame(client.get_historical_klines(symbol, time_frame, '1000 hours ago UTC'))

    df = df.iloc[:,0:6]
    df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    df.set_index('Time', inplace=True)
    df.index = pd.to_datetime(df.index, unit='ms')
    df = df.astype(float)
    return df

In [16]:
symbol = 'SOLUSDT'
data = get_data(symbol)

In [17]:
data.head()

,Open,High,Low,Close,Volume
Time,,,,,
2023-01-07 00:46:00,13.40,13.40,13.38,13.38,4757.67
2023-01-07 00:47:00,13.38,13.38,13.37,13.37,958.75
2023-01-07 00:48:00,13.37,13.41,13.37,13.39,7796.18
2023-01-07 00:49:00,13.39,13.40,13.38,13.38,229.16
2023-01-07 00:50:00,13.39,13.39,13.38,13.39,1626.74


In [20]:
def indicators_calc(df):
    for i in range(2, 10, 2):
        df[f'RSI_{i}'] = ta.momentum.rsi(df['Close'], window=i)
        df[f'SMA_{i*10}'] = df['Close'].rolling(i*10).mean()
    df['MACD'] = ta.trend.macd_diff(df['Close'])
    df.dropna(inplace=True)
    return df

In [21]:
data = indicators_calc(data)

In [22]:
data

,Open,High,Low,Close,Volume,RSI_2,SMA_20,RSI_4,SMA_40,RSI_6,SMA_60,RSI_8,SMA_80,MACD
Time,,,,,,,,,,,,,,
2023-01-07 02:05:00,13.33,13.34,13.33,13.33,75.71,27.567724,13.3230,47.192525,13.33225,50.251230,13.340667,50.639253,13.359875,0.002378
2023-01-07 02:06:00,13.34,13.35,13.33,13.33,684.32,27.567724,13.3235,47.192525,13.33250,50.251230,13.339500,50.639253,13.359250,0.002007
2023-01-07 02:07:00,13.33,13.34,13.32,13.32,1005.85,4.650285,13.3235,28.835499,13.33250,38.299890,13.338167,41.885669,13.358625,0.001061
2023-01-07 02:08:00,13.32,13.33,13.31,13.31,457.85,1.746502,13.3235,18.987685,13.33250,29.796130,13.336667,34.975955,13.357625,-0.000199
2023-01-07 02:09:00,13.32,13.32,13.31,13.31,435.90,1.746502,13.3235,18.987685,13.33225,29.796130,13.335500,34.975955,13.356750,-0.000951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-17 16:42:00,22.77,22.78,22.74,22.75,1771.32,42.333533,22.7105,59.092935,22.77350,58.912668,22.785000,56.701593,22.793625,0.014821
2023-02-17 16:43:00,22.75,22.77,22.75,22.76,700.42,57.823012,22.7100,63.501319,22.77275,61.662872,22.783000,58.830776,22.793000,0.014806
2023-02-17 16:44:00,22.75,22.80,22.75,22.79,5147.71,83.850306,22.7120,74.495411,22.77275,69.107080,22.782167,64.770435,22.793000,0.016138
